In [ ]:
import warnings
warnings.filterwarnings('ignore') 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

# RLAC MODEL
from sklearn.random_projection import GaussianRandomProjection
from scipy.stats import gaussian_kde
from scipy.signal import find_peaks  
from sklearn.neighbors import KernelDensity  
from sklearn.model_selection import GridSearchCV, LeaveOneOut
from scipy import stats
import diptest
from scipy.special import eval_hermitenorm  # For normalized Hermite polynomials H_n(x)
from scipy.stats import skew
from scipy.stats import norm
from sklearn.metrics import (adjusted_mutual_info_score, adjusted_rand_score, 
                             homogeneity_score, completeness_score, v_measure_score,
                             fowlkes_mallows_score, silhouette_score, calinski_harabasz_score,
                             davies_bouldin_score)

In [ ]:
import pandas as pd
import sc_loader as scl  

#  LOAD DATA 
sra_path = r'GSE41265 data\SraRunTable (1).csv'
tpm_path = r'GSE41265 data\GSE41265_allGenesTPM.txt.gz'
umb_path = r'GSE41265 data\GSE41265_umbExp.txt.gz'

# Load Main TPM
tpm_main_df = pd.read_csv(tpm_path, sep='\t', compression='gzip', index_col='GENE')

# Load UMB (Specific complex logic kept in notebook)
try:
    temp_umb = pd.read_csv(umb_path, sep='\t', compression='gzip', skiprows=[0,1], header=0)
    cols = temp_umb.columns.tolist()
    tpm_mb_df = temp_umb.set_index(cols[0])
    tpm_mb_df.index.name = 'GENE'
    tpm_mb_df = tpm_mb_df[[cols[1], cols[2], cols[3]]].copy()
    tpm_mb_df.columns = ['MB_S1', 'MB_S2', 'MB_S3']
except Exception as e:
    print(f"UMB Loading Error: {e}")
    tpm_mb_df = pd.DataFrame()

# RENAME (Dataset Specific)
main_map = {f'S{i}': f'GSM{1012776 + i}' for i in range(1, 19)}
main_map.update({f'P{i}': f'GSM{1012794 + i}' for i in range(1, 4)})
tpm_main_df = tpm_main_df.rename(columns=main_map)

mb_map = {'MB_S1': 'GSM1110889', 'MB_S2': 'GSM1110890', 'MB_S3': 'GSM1110891'}
tpm_mb_df = tpm_mb_df.rename(columns=mb_map)

#  PROCESS 
# Merge, Note: passing them as a list [df1, df2]
combined_tpm_df = scl.merge_expression_tables([tpm_main_df, tpm_mb_df])

# QC Visualization
print("\n--- Standard QC (Threshold > 0) ---")
lib_sizes, n_genes = scl.plot_qc_metrics(combined_tpm_df, detection_threshold=0)

print("\n--- Stricter QC (Threshold > 1) ---")
_ = scl.plot_qc_metrics(combined_tpm_df, detection_threshold=1)

In [ ]:
# 1. PROFESSIONAL IMPORT SETUP
# Get the current notebook directory
current_dir = os.getcwd()

# Define path to the shared_utils folder (one level up)
shared_path = os.path.abspath(os.path.join(current_dir, '..', 'shared_utils'))

# Add to system path if not already there
if shared_path not in sys.path:
    sys.path.append(shared_path)

import sc_processor as scp

In [ ]:
#  DEFINE PARAMETERS
# Centralizing these makes it easy to experiment with thresholds
QC_CONFIG = {
    'min_tpm': 2,
    'min_genes_per_sample': 3500,
    'min_samples_per_gene': 3
}

#  QC & FILTERING
# Unpack the tuple: filtered dataframe AND metrics for inspection
filtered_df, qc_metrics = scp.filter_tpm_matrix(
    combined_tpm_df, 
    **QC_CONFIG  # Automatically passes your specific parameters
)

# Inspect the QC metrics
print(qc_metrics.head())

#  NORMALIZATION
if not filtered_df.empty:
    log_tpm_df = scp.log_transform(filtered_df, method='log1p')
    
    # Visualization: Check if data looks normal-ish
    scp.plot_expression_distribution(
        log_tpm_df, 
        title="Distribution of Mean log(1+TPM) (Final Processed)"
    )
else:
    print("Skipping normalization: No data passed filtering.")

#  FEATURE SELECTION (HVG)
# We use the log-transformed data for this
df_hvg, hvg_metrics = scp.select_highly_variable_genes(
    log_tpm_df, 
    n_top_genes=2000
)

# Visual check of Dispersion
scp.plot_hvg_dispersion(hvg_metrics)

#  SCALING & PCA
# Scale the data (Z-score)
df_scaled = scp.scale_data(df_hvg)

# Run PCA
df_pca, var_ratio, pca_model = scp.run_pca_pipeline(df_scaled, n_components=50)

#  RESULTS VISUALIZATION
scp.plot_pca_results(df_pca, var_ratio)

# Check the final dataframe
print(df_pca.head())

In [ ]:
import sc_metadata as sc_meta 

# 1. LOAD SRA TABLE
# (Note: sra_path was defined in Step 1). We load it here to ensure we have the raw metadata
sra_df = pd.read_csv(sra_path)

# 2. PROCESS METADATA
# We use the outputs from the previous processing steps:
# qc_metrics -> from filter_tpm_matrix
# df_pca     -> from run_pca_pipeline

if not sra_df.empty and 'qc_metrics' in locals():
    
    final_cell_metadata_df = sc_meta.prepare_sample_metadata(
        sra_run_table_df=sra_df, 
        qc_metrics_df=qc_metrics, 
        pca_df=df_pca  # Use the PCA dataframe to define the final valid cells
    )
    # 3. INSPECT RESULTS
    if not final_cell_metadata_df.empty:
        col_to_check = 'Combined_Label'
        
        if col_to_check in final_cell_metadata_df.columns:
            print(f"\n--- Distribution of {col_to_check} ---")
            # Show counts of biological conditions
            print(final_cell_metadata_df[col_to_check].value_counts(dropna=False))
            
            # Optional: detailed breakdown
            print("\n--- Protocol Detail ---")
            print(final_cell_metadata_df['Protocol_Detail'].value_counts())
            
        # Preview the table
        print("\n--- Metadata Preview ---")
        print(final_cell_metadata_df.head())
    else:
        print("Metadata dataframe is empty.")
else:
    print("Pre-requisite DataFrames (SRA or QC Metrics) are missing.")

In [ ]:
# 1. PROFESSIONAL IMPORT SETUP
# Get the current notebook directory
current_dir = os.getcwd()

# Define path to the shared_utils folder (one level up)
shared_path = os.path.abspath(os.path.join(current_dir, '..', 'shared_utils'))

# Add to system path if not already there
if shared_path not in sys.path:
    sys.path.append(shared_path)

import sc_clustering as sc_cluster 

In [ ]:
#  CLUSTERING BENCHMARK
if 'df_pca' in locals() and 'final_cell_metadata_df' in locals():
    
    # Define the K values you want to test
    # Since we know there are roughly 4 biological conditions, we test around that number
    k_range = [3, 4, 5]

    clustering_results = sc_cluster.run_baseline_clustering(
        pca_df=df_pca, 
        cell_metadata=final_cell_metadata_df, 
        n_clusters_range=k_range,
        true_label_col='Combined_Label'
    )
    # --- DISPLAY LEADERBOARD ---
    if not clustering_results.empty:
        print("\n=== Final Clustering Leaderboard ===")
        print(clustering_results)
        
        # Optional: Select the best method/k for downstream analysis
        best_method = clustering_results.iloc[0]
        print(f"\nBest performing method: {best_method['Method']} (k={best_method['k']})")
        
else:
    print("Required data (PCA or Metadata) is missing. Please run previous cells.")

In [ ]:
import sys
import os

# 1. Get the path of the parent directory (Project_Root)
# '..' means "go up one level"
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# 2. Add it to Python's search path if not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# 3. Now you can import normally
from rlac import RLAC
from mdh import MDH

print("Successfully imported models from:", parent_dir)

In [ ]:
# --- 0. DATA SETUP ---
# Ensure we are using the Single Cell PCA data
if 'pca_data_df' not in locals() or 'final_cell_metadata_df' not in locals():
    raise NameError("CRITICAL: pca_data_df or final_cell_metadata_df is missing. Run previous cells.")

X_train = pca_data_df
y_train = final_cell_metadata_df['Combined_Label']

import warnings
import pandas as pd
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

# Import your custom models
from rlac import RLAC
from mdh import MDH

# --- CONFIGURATION ---
n_clusters = len(set(y_train))

# RLAC Parameters
rlac_methods = [
    'depth_ratio', 'dip', 'holes', 'min_kurt', 'max_kurt', 
    'negentropy', 'skewness', 'fisher', 'hermite', 'friedman_tukey'
]
rlac_params = {
    'random_state': [32, 42, 43, 44, 45],
    'bw_adjust': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
    'r': [None, 50, 100, 200, 300, 500] 
}

# MDH Parameters
mdh_config = {
    "h_multiplier": 1.0,
    "alphamax_val": 0.9,
    "alpha_steps": 5,
    "random_state": 42
}

results = []

print(f"\nStarting Benchmark on Single Cell Data (n={len(X_train)}, k={n_clusters})...")
print(f"\nTotal RLAC Combinations: {len(rlac_methods) * len(rlac_params['r']) * len(rlac_params['bw_adjust']) * len(rlac_params['random_state'])}")
print("-" * 80)

# ==========================================
# 1. RLAC LOOP
# ==========================================
for method in rlac_methods:
    for r_val in rlac_params['r']:
        for bw in rlac_params['bw_adjust']:
            for seed in rlac_params['random_state']:
                
                param_str = f"r={r_val}, bw={bw}, s={seed}"
                # Using end="" to keep the line until 'Done' is printed
                print(f"\n\rRunning RLAC {method:<15} | {param_str} ... ", end="")
                
                try:
                    model = RLAC(
                        n_clusters=n_clusters,
                        method=method,
                        r=r_val,
                        bw_adjust=bw,
                        random_state=seed,
                        plot=False
                    )
                    
                    with warnings.catch_warnings():
                        warnings.filterwarnings("ignore", category=UserWarning)
                        model.fit(X_train)
                    
                    ami = adjusted_mutual_info_score(y_train, model.labels_)
                    ari = adjusted_rand_score(y_train, model.labels_)
                    
                    results.append({
                        'Model': 'RLAC',
                        'Method': method,
                        'Params': param_str,
                        'AMI': ami,
                        'ARI': ari
                    })
                    
                except Exception as e:
                    # Fail silently in the loop to keep output clean, but record failure
                    results.append({
                        'Model': 'RLAC', 'Method': method, 'Params': param_str,
                        'AMI': -1, 'ARI': -1
                    })

print("\nRLAC Loop Complete.")

# ==========================================
# 2. MDH RUN
# ==========================================
print(f"\nRunning MDH {'Standard':<15} | h=1.0, a=0.9 ... ")
try:
    mdh_model = MDH(
        n_clusters=n_clusters,
        h_multiplier=mdh_config['h_multiplier'],
        alphamax_val=mdh_config['alphamax_val'],
        alpha_steps=mdh_config['alpha_steps'],
        random_state=mdh_config['random_state'],
        verbose=False,
        plot=False
    )
    
    mdh_model.fit(X_train)
    
    ami_mdh = adjusted_mutual_info_score(y_train, mdh_model.labels_)
    ari_mdh = adjusted_rand_score(y_train, mdh_model.labels_)
    
    print(f"Done (AMI: {ami_mdh:.4f})")
    
    results.append({
        'Model': 'MDH',
        'Method': 'Standard',
        'Params': 'Fixed',
        'AMI': ami_mdh,
        'ARI': ari_mdh
    })
    
except Exception as e:
    print(f"MDH FAILED. Error: {e}")

# ==========================================
# 3. RESULTS TABLE
# ==========================================
print("\n" + "="*80)
print("FINAL RESULTS (ALL MODELS - SORTED BY AMI)")
print("="*80)

# Create DataFrame and sort
results_df = pd.DataFrame(results).sort_values(by='AMI', ascending=False)

# Print everything
print(results_df.to_string(index=False))

In [ ]:
================================================================================
FINAL RESULTS (ALL MODELS - SORTED BY AMI)
================================================================================
Model         Method                Params       AMI       ARI
 RLAC        hermite   r=100, bw=0.3, s=45  0.583942  0.425532
 RLAC            dip   r=100, bw=0.2, s=45  0.583942  0.425532
 RLAC    depth_ratio   r=500, bw=0.1, s=43  0.583942  0.425532
 RLAC        hermite   r=100, bw=0.1, s=45  0.583942  0.425532
 RLAC        hermite   r=100, bw=0.2, s=45  0.583942  0.425532
 RLAC    depth_ratio   r=500, bw=0.5, s=43  0.583942  0.425532
 RLAC    depth_ratio   r=300, bw=0.4, s=45  0.583942  0.425532
 RLAC         fisher   r=100, bw=0.2, s=45  0.583942  0.425532
 RLAC        hermite   r=100, bw=0.4, s=45  0.583942  0.425532
 RLAC         fisher   r=100, bw=0.4, s=45  0.583942  0.425532
 RLAC        hermite  r=100, bw=0.05, s=45  0.583942  0.425532
 RLAC    depth_ratio   r=500, bw=0.3, s=43  0.583942  0.425532
 RLAC    depth_ratio   r=300, bw=0.2, s=45  0.583942  0.425532
